The Intelligence of Dogs

In [ ]:
%load_ext nb_black

from collections import namedtuple
import pathlib
from pyquery import PyQuery as pq
from urllib import parse

# Wikipedia en domain.
WIKI = 'https://en.wikipedia.org'

# Intelligence tier rank thresholds.
TIER_THRESH = [10, 31, 62, 104, 127, 138]

# Breed of the dog.
Breed = namedtuple('Breed', 'rank, name, tier, wiki, img, raw, proc')

# Returns PyQuery object for wiki path p.
def pq_wiki(p):
    return pq(url=parse.urljoin(WIKI, p))

# Returns the img src of the infobox image.
def new_breed(rank, wiki):
    doc = pq_wiki(wiki)
    name = doc('#firstHeading').text()
    img = doc('.infobox-image>a>img')[0].attrib['src']
    fname = '{:0>3d}_{}'.format(rank, name.replace(' ', '_'))
    raw = '{}{}'.format(fname, pathlib.Path(img).suffix)
    proc = '{}.jpg'.format(fname)
    tier = 0
    for i, thresh in enumerate(TIER_THRESH):
        if (rank <= thresh):
            tier = len(TIER_THRESH) - i
            break
    return Breed(rank=rank, name=name, tier=tier, wiki=wiki, img=img, raw=raw, proc=proc)

Read/Write breeds and raw images

In [ ]:
import requests
import simplejson

def write_breeds(breeds):
    f = open('breeds.json', 'w')
    f.write(simplejson.dumps(breeds, indent=4*' '))
    f.close()

def read_breeds():
    f = open('breeds.json', 'r')
    breeds = [Breed(**d) for d in simplejson.loads(f.read())]
    f.close()
    return breeds

def download_raw(breeds):
    for breed in breeds:
        f = open('raw/{}'.format(breed.raw), "wb")
        f.write(requests.get('https:{}'.format(breed.img)).content)
        f.close()

In [ ]:
# Fetch and parse the page.
doc = pq_wiki('/wiki/The_Intelligence_of_Dogs')

# Add 1 to the index to get rank.
# Some wierd reason [17, 130] rank is [Collie, Mastiff] which is a type of dog. Skipping.
skip = frozenset([17, 130])
breeds = [new_breed(i+1, a.attrib['href']) for i, a in enumerate(doc('ol>li>a')) if i+1 not in skip]

# Download raw images.
download_raw(breeds)

In [ ]:
write_breeds(breeds)
# Just testing the read.
breeds = read_breeds()

Add frame to the images

In [ ]:
from collections import Counter
import cv2 as cv
import numpy as np
import random
import pathlib
import shutil
from sklearn.cluster import KMeans

# Returns BGR color.
def color(color):
    rgb = color[1:]
    return [int(rgb[4:6], 16), int(rgb[2:4], 16), int(rgb[:2], 16)]

# Scanner Darkly: red, oranga, yellow, green, blue, violet
PALETTE = {1: color('#912318'), 2: color('#EE5D02'), 3: color('#E5ED3C'), 4: color('#80A82E'), 5:  color('#496BBC'), 6: color('#3B3559')}
SIZE = 500
BORDER = 32

# Read the raw image and scale to size.
def read_breed_raw(raw, size):
    image = cv.imread('raw/{}'.format(raw), cv.IMREAD_COLOR)
    scale = size/max(image.shape)
    dim = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    return cv.resize(image, dim, interpolation=cv.INTER_LANCZOS4)

# Add Scanner Darkly like effect.
def scanner_darkly(image, blur, n_clusters, ribbon):
    # Equalize histogra for Y.
    image = cv.cvtColor(image, cv.COLOR_BGR2YUV)
    image[:, :, 0] = cv.equalizeHist(image[:, :, 0])
    image = cv.cvtColor(image, cv.COLOR_YUV2BGR)

    # Smooth image.
    image = cv.GaussianBlur(image, blur, cv.BORDER_WRAP)
    (h, w) = image.shape[:2]
    
    # Quantize colors.
    image = cv.cvtColor(image, cv.COLOR_BGR2LAB)
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    clt = KMeans(n_clusters = n_clusters)
    labels = clt.fit_predict(image)
    quant = clt.cluster_centers_.astype('uint8')[labels]

    # Add least common color as the background.
    hist = Counter(labels).most_common()
    labels = np.full((SIZE, SIZE), hist[n_clusters-1][0])
    bg = clt.cluster_centers_.astype('uint8')[labels]
    bg = bg.reshape((SIZE, SIZE, 3))
    bg = cv.cvtColor(bg, cv.COLOR_LAB2BGR)

    # Add the frame.
    yoff = round(w/3)
    xoff = round(h/3)
    bg[yoff:yoff+BORDER, 0:SIZE] = np.full((BORDER, SIZE, 3), ribbon)
    bg[0:SIZE, xoff:xoff+BORDER] = np.full((SIZE, BORDER, 3), ribbon)

    # Write quantized image.
    quant = quant.reshape((h, w, 3))
    quant = cv.cvtColor(quant, cv.COLOR_LAB2BGR)

    yoff = round((SIZE-h)/2)
    xoff = round((SIZE-w)/2)
    bg[yoff:yoff+h, xoff:xoff+w] = quant
    
    return bg

# Process all breeds.
def proc_breeds(breeds, num):
    shutil.rmtree('proc')
    pathlib.Path('proc').mkdir()
    for breed in random.sample(breeds, num):
        image = read_breed_raw(breed.raw, 420)
        proc = scanner_darkly(image, (5, 5), 12, PALETTE[breed.tier])
        cv.imwrite('proc/{}'.format(breed.proc), proc)

In [ ]:
proc_breeds(breeds, len(breeds))

Add Logo

In [ ]:
def create_logo():
    doc = pq_wiki('/wiki/Paw')
    img = doc('.thumbinner>a>img')[0].attrib['src']
    f = open('logo/raw.jpg', "wb")
    f.write(requests.get('https:{}'.format(img)).content)
    f.close()
    image = cv.imread('logo/raw.jpg', cv.IMREAD_COLOR)
    scale = 420/max(image.shape)
    dim = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    image = cv.resize(image, dim, interpolation=cv.INTER_LANCZOS4)
    proc = scanner_darkly(image, (5, 5), 12, [0, 0, 0])
    cv.imwrite('logo/proc.jpg', proc)
    logo_json = {'name': 'The Intelligence of Dogs', 'wiki': '/wiki/The_Intelligence_of_Dogs', 'img': img}
    f = open('logo/meta.json', "w")
    f.write(simplejson.dumps(logo_json, indent=4*' '))
    f.close()

In [ ]:
create_logo()

Add Background

In [ ]:
def create_bg():
    doc = pq_wiki('/wiki/A_Scanner_Darkly_(film)')
    img = doc('.infobox-image>a>img')[0].attrib['src']
    f = open('bg/raw.jpg', "wb")
    f.write(requests.get('https:{}'.format(img)).content)
    f.close()
    image = cv.imread('bg/raw.jpg', cv.IMREAD_COLOR)
    scale = 420/max(image.shape)
    dim = (int(image.shape[1] * scale), int(image.shape[0] * scale))
    image = cv.resize(image, dim, interpolation=cv.INTER_LANCZOS4)
    proc = scanner_darkly(image, (5, 5), 12, [255, 255, 255])
    cv.imwrite('bg/proc.jpg', proc)
    logo_json = {'name': 'A Scanner Darkly (film)', 'wiki': '/wiki/A_Scanner_Darkly_(film)', 'img': img}
    f = open('bg/meta.json', "w")
    f.write(simplejson.dumps(logo_json, indent=4*' '))
    f.close()

In [ ]:
create_bg()

Fetch Assets from OpenSea

In [ ]:
import cv2 as cv
import pathlib
import requests
import shutil
import simplejson

CONTRACT = '0x495f947276749ce646f68ac8c248420045cb7b5e'
OWNER = '0xdC58Ea900B5FE2502947eeA686C9A485D4a58684'

# TODO: replace above.
def write_json(p, data):
    f = open(p, 'w')
    f.write(simplejson.dumps(data, indent=4*' '))
    f.close()

# TODO: replace above.
def write_binary(p, src):
    f = open(p, "wb")
    f.write(requests.get(src).content)
    f.close()

def download_opensea():
    shutil.rmtree('opensea')
    pathlib.Path('opensea').mkdir()

    # Download collection json.
    response = requests.request('GET', 'https://api.opensea.io/api/v1/collections', params={'asset_owner': OWNER})
    j = simplejson.loads(response.text)[0]
    write_json('opensea/{}.json'.format(j['name']), j)

    # Download all assets.
    assets = []
    for offset in [0, 50, 100]:
        response = requests.request('GET', 'https://api.opensea.io/api/v1/assets', params={'offset': '{}'.format(offset), 'limit': '50', 'collection': j['slug']})
        assets.extend(simplejson.loads(response.text)['assets'])

    # Downloadd images and store it along with the json.
    # 4, 16 and 26 are all redirecting to Belgian Shephard, only minting 14.
    # 42. and 69 both Australian Shephard, only minting 42.
    # 17, 72and 130 Collie, Pointer, Mastiff are types, skipping.
    skip = frozenset({16, 17, 26, 69, 72, 130})
    missing = set(range(1, 139))
    props = {}
    for asset in assets:
        rank = int([trait['value'] for trait in asset['traits'] if trait['trait_type'] == 'rank'][0])
        desc = simplejson.loads(asset['description'].replace('```', ''))
        assert rank == desc['rank'], 'rank mismatch'

        tier = [trait['value'] for trait in asset['traits'] if trait['trait_type'] == 'tier'][0]
        assert tier.startswith('{}'.format(desc['tier'])), '{}: tier {} does not startwith {}'.format(rank, tier, desc['tier'])

        fname = '{:0>3d}_{}'.format(rank, desc['name'].replace(' ', '_'))
        assert pathlib.Path('proc/{}.jpg'.format(fname)).is_file(), '{}: {} does not exist'.format(rank, 'proc/{}.jpg'.format(fname))

        # Write asset.
        pathlib.Path('opensea/{}'.format(fname)).mkdir()
        write_json('opensea/{}/asset.json'.format(fname), asset)

        # Write images.
        write_binary('opensea/{}/image.jpg'.format(fname), asset['image_url'])
        write_binary('opensea/{}/image_preview.jpg'.format(fname), asset['image_preview_url'])
        write_binary('opensea/{}/image_thumbnail.jpg'.format(fname), asset['image_thumbnail_url'])

        image = cv.imread('opensea/{}/image.jpg'.format(fname), cv.IMREAD_COLOR)
        props[rank] = {'fname': fname, 'bgr': image[0][0].tolist()}
        missing.remove(rank)

    # Write props.
    assert missing == skip, '{} != {}'.format(missing, skip)
    write_json('opensea/props.json', props)

In [ ]:
download_opensea()

Collage: 35 x 35 px tiles, 12 x 12 collage

In [ ]:
from collections import Counter
import cv2 as cv
import math
import numpy as np
import simplejson
from sklearn.cluster import KMeans

TILE_SIZE = 35
TILE_NUM = 12
COLLAGE_SIZE = TILE_NUM * TILE_SIZE
TOTAL = 138
BORDER = 5

def create_collage(n_clusters):
    f = open('opensea/props.json', 'r')
    props = simplejson.loads(f.read())
    f.close()

    # read images by rank.
    colors = np.empty((0, 3))
    logocolor = cv.imread('logo/proc.jpg', cv.IMREAD_COLOR)[0][0]
    bgcolor = cv.imread('bg/proc.jpg', cv.IMREAD_COLOR)[0][0]
    for (rank, prop) in props.items():
        image = cv.cvtColor(np.full((1, 1, 3), np.uint8(prop['bgr'])), cv.COLOR_BGR2LAB)
        image = image.reshape(1, 3)
        colors = np.concatenate((colors, image))
    colors = np.concatenate((colors, [bgcolor, logocolor]))
    
    clt = KMeans(n_clusters=n_clusters)
    labels = clt.fit_predict(colors)
    hist = Counter(labels).most_common()
    labels = np.full((COLLAGE_SIZE, COLLAGE_SIZE), hist[n_clusters - 1][0])

    # Make collage background.
    collage = clt.cluster_centers_.astype('uint8')[labels]
    collage = collage.reshape((COLLAGE_SIZE, COLLAGE_SIZE, 3))
    raw = cv.cvtColor(collage, cv.COLOR_LAB2BGR)
    bg = np.copy(raw[0][0])
    rank = 0
    coord = [0, 0]
    for y in range(TILE_NUM):
        e = (0 if y % 2 == 0 else TILE_NUM - 1)
        d = (1 if y % 2 == 0 else -1)
        for x in range(TILE_NUM):
            coord = (e + x * d, y)
            rank = rank + 1
            if str(rank) in props:
                prop = props[str(rank)]
                image = np.full((TILE_SIZE, TILE_SIZE, 3), prop['bgr'])
            else:
                image = np.full((TILE_SIZE, TILE_SIZE, 3), bg)
            yoff = coord[1] * TILE_SIZE
            xoff = coord[0] * TILE_SIZE
            raw[yoff:yoff + TILE_SIZE, xoff:xoff + TILE_SIZE] = image

    image = np.full((TILE_SIZE, TILE_SIZE, 3), logocolor)
    yoff = (TILE_NUM - 1) * TILE_SIZE
    raw[yoff:yoff + TILE_SIZE, 0:TILE_SIZE] = image

    image = np.full((TILE_SIZE, TILE_SIZE, 3), bgcolor)
    xoff = TILE_SIZE
    raw[yoff:yoff + TILE_SIZE, xoff:xoff + TILE_SIZE] = image
    cv.imwrite('collage/raw.jpg', raw)

    image = np.full((SIZE, SIZE, 3), bg)
    yoff = round(COLLAGE_SIZE/3)
    xoff = round(COLLAGE_SIZE/3)
    for i, color in PALETTE.items():
        # Add the frame.
        image[yoff:yoff+BORDER, 0:SIZE] = np.full((BORDER, SIZE, 3), color)
        image[0:SIZE, xoff:xoff+BORDER] = np.full((SIZE, BORDER, 3), color)
        yoff += BORDER
        xoff += BORDER
    image[yoff:yoff+2, 0:SIZE] = np.full((2, SIZE, 3), [0, 0, 0])
    image[0:SIZE, xoff:xoff+2] = np.full((SIZE, 2, 3), [0, 0, 0])
    image[40:460, 40:460] = raw
    cv.imwrite('collage/proc.jpg', image)

In [ ]:
from matplotlib import pyplot as plt

create_collage(n_clusters=12)

Add config json

In [ ]:
from collections import OrderedDict
import cv2 as cv
import simplejson

def read_json(p):
    f = open(p, 'r')
    j = simplejson.loads(f.read())
    f.close()
    return j

def create_config():
    props = read_json('opensea/props.json')
    breeds = {breed.rank:breed for breed in read_breeds()}
    breeds = [(breeds[int(k)], v) for k,v in props.items()]

    def get_token(fname):
        j = read_json('opensea/{}/asset.json'.format(fname))
        return (j['id'], j['token_id'], j['name'])

    tiles = [{
        'rank': b.rank, 
        'name': b.name, 
        'tier': b.tier, 
        'wiki': b.wiki, 
        'img': b.img,
        'id': get_token(p['fname']),
        'fname': p['fname'], 
        'bgr': p['bgr']} for (b, p) in breeds]
    
    def add_tile(rank, name, tier, fname, p, color):
        j =read_json(p)
        tile = {
            'rank': rank, 
            'name': name, 
            'tier': tier, 
            'wiki': j['wiki'],
            'img': j['img'], 
            'fname': fname, 
            'bgr': color}
        tiles.append(tile)
    add_tile(143, 'Cover', 0, 'bg', 'bg/meta.json', cv.imread('bg/proc.jpg', cv.IMREAD_COLOR)[0][0].tolist())
    add_tile(144, 'ScannerBarkly\'s Stamp', 7, 'logo', 'logo/meta.json', cv.imread('logo/proc.jpg', cv.IMREAD_COLOR)[0][0].tolist())
    tiles = sorted(tiles, key=lambda x: x['rank'])
    collagecolor = cv.imread('collage/proc.jpg', cv.IMREAD_COLOR)[0][0].tolist() 
    colors = {'bg': collagecolor, '0': [255, 255, 255], '7': [0, 0, 0]}
    colors.update({str(k): v for k, v in PALETTE.items()})
    site = {'contract': CONTRACT, 'slug': 'scannerbarkly', 'tiles': tiles, 'colors': OrderedDict(sorted(colors.items()))}
    f = open('config.json', 'w')
    f.write(simplejson.dumps(site, indent=4*' '))
    f.close()

In [ ]:
create_config()

Add QR code

In [55]:
import numpy as np
import qrcode

def write_qr(url, bg, p):
    qr = qrcode.QRCode(
        version=16,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        border=0,
        box_size=6,
    )
    qr.add_data(url)
    qr.make(fit=True)
    image = qr.make_image(fill_color="black", back_color="white")
    image.save(p)
    bg = cv.imread(bg, cv.IMREAD_COLOR)
    fg = np.full((SIZE, SIZE, 3), [0, 0, 0])
    fg[7:493, 7:493] = cv.imread(p , cv.IMREAD_COLOR)
    fg = np.where(fg == 0, bg, fg)
    cv.imwrite(p, fg)
    return

<IPython.core.display.Javascript object>

In [56]:
write_qr('https://scannerbarkly.art/#/','collage/proc.jpg', 'collage/qr.jpg')

<IPython.core.display.Javascript object>

Debug

In [ ]:
from matplotlib import pyplot as plt
import pathlib

f = plt.figure(figsize=(30,180))
for i, p in enumerate(sorted(pathlib.Path('proc').iterdir())):
    image = cv.imread(str(p), cv.IMREAD_COLOR)
    f.add_subplot(30, 5, i+1)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(str(p))
    plt.axis('off')
plt.show()

In [ ]:
import json

def price(rank):
    return '{:0.4f}'.format(1/rank)

def pdesc(rank):
    j = json.loads(simplejson.dumps(breeds[rank-1]))
    del j['raw']
    del j['proc']
    print(j['name'])
    print('```\n{}\n```'.format(simplejson.dumps(j, indent=4*' ')))
    print(price(rank))